In [ ]:
!pip install konlpy

In [ ]:
!pip install -U pyLDAvis

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('샵다이소_구글앱_리뷰_정제.csv')
df.head()

In [ ]:
from konlpy.tag import Okt
import re

In [ ]:
def tokenize_text(text):
    text = re.sub(r"[^ㄱ-ㅣ가-힣\s]","",text)
    okt = Okt()
    okt_morphs = okt.pos(text)

    words = []
    for word,pos in okt_morphs: 
        if pos == 'Adjective' or pos=='Verb' or pos=='Noun':
            words.append(word)

    word_str =  ' '.join(words)  
    return word_str

In [ ]:
from tqdm import tqdm
token_list = []
for temp in tqdm(df['text']):
    token_list.append(tokenize_text(temp))
token_list

In [ ]:
drop_corpus = []

for index in range(len(token_list)):
    corpus = token_list[index]
    if len(set(corpus.split())) < 3:
        drop_corpus.append(corpus)

for corpus in drop_corpus:
    token_list.remove(corpus)

token_list

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
count_vec = CountVectorizer(max_df=0.2,max_features=1000,min_df=3,ngram_range=(1,2))
feat_vect = count_vec.fit_transform(token_list)
print(feat_vect.shape)
print(count_vec.vocabulary_)

In [ ]:
feature_names = count_vec.get_feature_names_out()

In [ ]:
lda = LatentDirichletAllocation(n_components=5,max_iter=20)
lda.fit(feat_vect)

In [ ]:
lda.components_

In [ ]:
def display_topics(model,feature_names,num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('토픽',topic_index)
        topic_word_indexes = topic.argsort()[::-1]
        top_index = topic_word_indexes[:num_top_words]
       
        f_name_list = []
        for temp in top_index:
            f_name_list.append(feature_names[temp])

        feature_concat = ' '.join(f_name_list)
        print(feature_concat)
    

In [ ]:
display_topics(lda,feature_names,15)

In [ ]:
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda,feat_vect,count_vec)
pyLDAvis.display(vis)

In [ ]:
sent_topic = lda.transform(feat_vect)
print(sent_topic[0])

In [ ]:
doc_per_topic_list = []
for n in range(sent_topic.shape[0]):
    topic_most_pr = sent_topic[n].argmax()
    topic_pr = sent_topic[n].max()
    doc_per_topic_list.append([n,topic_most_pr,topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list,columns=['no','토픽번호','확률'])
doc_topic_df



In [ ]:
for topic in range(len(doc_topic_df['토픽번호'].unique())):
    print('토픽',topic)
    top_topic = doc_topic_df[ doc_topic_df['토픽번호']==topic].sort_values(by='확률',ascending=False)
    print(df['text'].iloc[ top_topic['no'].iloc[0]])
    print(df['text'].iloc[ top_topic['no'].iloc[1]])
    print(df['text'].iloc[ top_topic['no'].iloc[2]])

# 분석 결과

**0**  
상품, 매장, 가입, 주문, 검색, 물건, 회원, 선택, 품절, 어플, 회원가입, 제품, 해도, 안됨, 취소, 하면, 결제, 해야, 삭제, 구매, 장바구니, 다이소, 해서, 없는, 하는, 옵션, 있는, 사항, 사용

**1**  
매장, 검색, 재고, 주소, 등록, 다이소, 배송지, 안됨, 입력, 지역, 없다고, 근처, 확인, 온라인, 서비스, 가능한, 매장 검색, 매장 재고, 배송지 입력, 되는, 없네요, 하나, 안되고, 배달, 배송지 등록, 안도는, 건지, 진짜, 상품

**2**  
다이소, 지역, 배달, 불가, 배송비, 가능, 고객, 제품, 물건, 불가 지역, 있는, 하는데, 설정, 상품, 주문, 이용, 있어서, 좋아요, 무슨, 저희, 구매, 다른, 해도, 경우, 어플, 서비스, 갑자기, 택배, 합니다, 배송 불가

**3**  
매장, 다이소, 지역, 픽업, 로그인, 아이디, 서비스, 실행, 사용, 어플, 다시, 진짜, 주변, 개선, 안됨, 이용, 안되는, 있는데, 계속, 군데, 오류, 하면, 그냥, 근처, 좋겠어요, 안되네요, 입력, 서비스 지역, 접속, 문의

**4**   
로그인, 지역, 어플, 오류, 연결, 인터넷, 계속, 화면, 인터넷 연결, 온라인, 사용, 서비스, 가능 지역, 하면, 설정, 가능, 가입, 안되고, 주소, 하지, 하는, 접속, 와이파이, 했는데, 회원 가입, 데이터, 문제, 최악, 해주세요, 제대로

-> 정확하지 않은 매장 결과과

#### **결론**

토픽 1,2,3,4,5에서 공통적으로 '안됨, 없네요, 안되고, 안되는, 불가, 배송 불가, 오류, 안되네요'와 같은 부정적인 키워드들을 볼 수 있으며, 이를 통해 앱에서 이용하고자하는 서비스가 제대로 되지 않는다는 것을 볼 수 있다.

1. 특정 지역 배송 안됨 : 토픽0, 토픽1, 토픽2, 토픽4
2. 어플 연결 오류: 토픽0, 토픽1, 토픽2, 토픽3, 
3. 특정 지역 매장에서 사용 불가 : 토픽0, 토픽2,  토픽4
4. 앱 서비스가 제대로 제공되지 않음
- 재고파악이 제대로 되지 않음 - 토픽4
- 픽업 주문이 제대로 실행되지 않음 - 토픽4
- 앱에서 주문 취소가 이루어지지 않음 - 토픽3



앱 개선 사항

- 재고 : 재고가 어느 정도 남았는지, 품절상태인지, 입고예정이 있는지 없는지

- 배송 : 배송지 확대, 옵션 선택

- 픽업 : 픽업이 원활히 진행될 수 있도록 각 매장별 관리 시스템 도입, 매장 픽업비(종이백 필수에서 그냥 들고가거나, 장바구니 사용)

- 특정 매장 픽업 서비스 불가 : 물류 센터에서 각 매장별 들어온 주문 상품을 전달함으로써..